In [1]:
from nats_bench import create
from tqdm import tqdm
import numpy as np

def close_to(a, b, eps=1e-4):
    if b != 0 and abs(a-b) / abs(b) > eps:
        return False
    if a != 0 and abs(a-b) / abs(a) > eps:
        return False
    return True

In [2]:
def check_flops_params(xapi):
    print(f"Check {xapi}")
    datasets = ("cifar10-valid", "cifar10", "cifar100", "ImageNet16-120")
    counts = 0
    for index in tqdm(range(len(xapi))):
        for dataset in datasets:
            info_12 = api.get_cost_info(index, dataset, hp="12")
            info_full = api.get_cost_info(index, dataset, hp=xapi.full_epochs_in_paper)
            assert close_to(info_12['flops'], info_full['flops']), f"The {index}-th " \
            f"architecture has issues on {dataset} " \
            f"-- {info_12['flops']} vs {info_full['flops']}."  # check the FLOPs
            assert close_to(info_12['params'], info_full['params']), f"The {index}-th " \
            f"architecture has issues on {dataset} " \
            f"-- {info_12['params']} vs {info_full['params']}."  # check the number of parameters
            counts += 1
    print(f"Check {xapi} completed -- {counts} arch-dataset pairs.")

In [ ]:
# Create the API for size search space
api = create(None, 'sss', fast_mode=True, verbose=False)
print(f'There are {len(api)} architectures in the size search space -- {api}')
check_flops_params(api)

  0%|          | 3/32768 [00:00<19:31, 27.97it/s]

[2022-01-19 23:22:51] Try to use the default NATS-Bench (size) path from fast_mode=True and path=None.
There are 32768 architectures in the size search space -- NATSsize(0/32768 architectures, fast_mode=True, file=None)
Check NATSsize(0/32768 architectures, fast_mode=True, file=None)


  4%|▍         | 1350/32768 [00:43<16:55, 30.94it/s]